In [3]:
import pandas as pd
df = pd.read_csv('./datasets/16-09-23.csv')

In [4]:
df

,Packet ID,TIME,Size,eth.src,eth.dst,IP.src,IP.dst,IP.proto,port.src,port.dst
0,1,1474552802,70,18:b7:9e:02:20:44,14:cc:20:51:33:ea,192.168.1.120,46.105.38.79,6,40234,5228
1,2,1474552802,66,18:b7:9e:02:20:44,14:cc:20:51:33:ea,192.168.1.120,46.105.38.79,6,40234,5228
2,3,1474552802,66,14:cc:20:51:33:ea,30:8c:fb:2f:e4:b2,52.87.241.159,192.168.1.106,6,443,46330
3,4,1474552802,60,d0:52:a8:00:67:5e,14:cc:20:51:33:ea,192.168.1.196,52.1.40.122,6,3063,443
4,6,1474552802,88,70:ee:50:18:34:43,14:cc:20:51:33:ea,192.168.1.241,192.168.1.1,17,61725,53
...,...,...,...,...,...,...,...,...,...,...
802576,947066,1474639198,95,44:65:0d:56:cc:d3,14:cc:20:51:33:ea,192.168.1.240,205.251.242.52,6,40184,443
802577,947067,1474639198,66,14:cc:20:51:33:ea,30:8c:fb:2f:e4:b2,52.87.241.159,192.168.1.106,6,443,46330
802578,947068,1474639199,54,14:cc:20:51:33:ea,44:65:0d:56:cc:d3,205.251.242.52,192.168.1.240,6,443,40184
802579,947071,1474639199,156,30:8c:fb:2f:e4:b2,14:cc:20:51:33:ea,192.168.1.106,52.87.241.159,6,46330,443
